**Code to mount the data from Azure Blob storage to databricks

In [0]:
'''dbutils.fs.mount(
  source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net",
  mount_point = "/mnt/iotdata",
  extra_configs = {"fs.azure.account.key.<storage-account-name>.blob.core.windows.net":dbutils.secrets.get(scope = "<scope-name>", key = "<key-name>")})'''

step 1-Mounting the raw data from Blob storage of Azure to databricks for transformation

In [0]:
dbutils.fs.mount(
  source = "wasbs://container-pizzasales@storepizzasales.blob.core.windows.net",
  mount_point = "/mnt/container2",
  extra_configs = {"fs.azure.account.key.storepizzasales.blob.core.windows.net":"sXckuSZfkpg60GknwgzCWDbMwAzIj4j5If0jp0+oOcxbRpm8+is5R/gXo3CVe4wZnkA7FLbsmXA++AStIVe1jQ=="})

In [0]:
dbutils.fs.ls("/mnt/container2")

step 2--Reading the data and creating a dataframe

In [0]:
df= spark.read.format("csv").options(header='true',inferschema='true').load('dbfs:/mnt/container2/data_8c6eb94e-2d41-403a-915f-56a4fa670482_6a484f54-d490-4b9c-9bb3-2d558423e57e.txt')

Displaying the data

In [0]:
df.display()

step 3--Creating a temp view of the dataframe to conduct sql operations

In [0]:
df.createOrReplaceTempView("pizza_sales_analysis")

In [0]:
%sql
select * from pizza_sales_analysis;

Step 4--Creating a new dataset by doing some transformations and grouping of the required data for further analysis.

In [0]:
%sql
select
count(distinct order_id) as order_id,
sum(quantity)as quantity,
date_format(order_date,'MMM') as month_name,
date_format(order_date,'EEEE') as day_name,
hour(order_time) as order_time,
sum(unit_price) as unit_price,
sum(total_price) as total_price,
pizza_size,
pizza_category,
pizza_name
from pizza_sales_analysis
group by 3,4,5,8,9,10;

The next step is to use the grouped and transformed data for further analysis in Power bi and create a report